# About: VM - Destroy VM on KVM

---

VMの停止用 Notebook。

## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。ここでは、**VMを収容している物理マシンを指定する。**

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象マシンにlibvirtがインストールされているかを確認する。

In [3]:
!ansible -b -a 'virsh version' {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Compiled against library: libvirt 0.10.2
Using library: libvirt 0.10.2
Using API: QEMU 0.10.2
Running hypervisor: QEMU 0.12.1



# 停止対象のVM確認

停止したいVMのIPアドレスまたはグループ名を定義する。

In [4]:
target_vm = 'test-vms'

対象にpingが通ることを確認する。

In [5]:
!ansible -m ping {target_vm}

XXX.XXX.XXX.71 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.70 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.112 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.113 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.114 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象のIPアドレスのリストを得る。

In [7]:
ping_stdout = !ansible -m ping {target_vm}
target_vm_addrs = map(lambda l: l.split()[0], filter(lambda l: 'SUCCESS' in l, ping_stdout))
target_vm_addrs

['XXX.XXX.XXX.71',
 'XXX.XXX.XXX.72',
 'XXX.XXX.XXX.70',
 'XXX.XXX.XXX.73',
 'XXX.XXX.XXX.112',
 'XXX.XXX.XXX.113',
 'XXX.XXX.XXX.114']

対象のVM名を検索する。

In [8]:
vmnames = !ansible -b -a 'virsh list' {target_group}
vmnames = vmnames[1:]
vmnames = vmnames[map(lambda l: l[0], filter(lambda l: l[1].startswith('----'), enumerate(vmnames)))[0] + 1:]
vmnames = map(lambda l: l.split()[1], vmnames)
vmnames

['vm1', 'vm2', 'vm3', 'vm4', 'vm5', 'vm6', 'vm7']

In [9]:
import re
def get_mac_address(vmname):
    domiflist_stdio = !ansible -b -a "virsh domiflist {vmname}" {target_group}
    mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
    vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
    return vmmac

def get_ip_address(vmmac):
    leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
    ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
    ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
    return ipaddr

In [10]:
vmdescs = zip(vmnames, map(lambda mac: get_ip_address(mac), map(lambda n: get_mac_address(n), vmnames)))
vmdescs

[('vm1', 'XXX.XXX.XXX.112'),
 ('vm2', 'XXX.XXX.XXX.113'),
 ('vm3', 'XXX.XXX.XXX.114'),
 ('vm4', 'XXX.XXX.XXX.70'),
 ('vm5', 'XXX.XXX.XXX.72'),
 ('vm6', 'XXX.XXX.XXX.71'),
 ('vm7', 'XXX.XXX.XXX.73')]

対象のVM名を取得する。

In [12]:
target_vmnames = map(lambda a: filter(lambda e: e[1] == a, vmdescs)[0], target_vm_addrs)
target_vmnames

[('vm6', 'XXX.XXX.XXX.71'),
 ('vm5', 'XXX.XXX.XXX.72'),
 ('vm4', 'XXX.XXX.XXX.70'),
 ('vm7', 'XXX.XXX.XXX.73'),
 ('vm1', 'XXX.XXX.XXX.112'),
 ('vm2', 'XXX.XXX.XXX.113'),
 ('vm3', 'XXX.XXX.XXX.114')]

VMの状態の確認。runningならば想定通り。

In [14]:
for vmname, vmaddr in target_vmnames:
    print('Status of {}'.format(vmaddr))
    !ansible -b -a "virsh domstate {vmname}" {target_group}

Status of XXX.XXX.XXX.71
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.72
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.70
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.73
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.112
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.113
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running

Status of XXX.XXX.XXX.114
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
running



# 仮想マシンの停止

仮想マシンを停止する。

In [15]:
for vmname, vmaddr in target_vmnames:
    print('Destroying {}...'.format(vmaddr))
    !ansible -b -a "virsh destroy {vmname}" {target_group}

Destroying XXX.XXX.XXX.71...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm6 destroyed

Destroying XXX.XXX.XXX.72...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm5 destroyed

Destroying XXX.XXX.XXX.70...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm4 destroyed

Destroying XXX.XXX.XXX.73...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm7 destroyed

Destroying XXX.XXX.XXX.112...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm1 destroyed

Destroying XXX.XXX.XXX.113...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm2 destroyed

Destroying XXX.XXX.XXX.114...
XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain vm3 destroyed



しばらく待ってから再度 virsh listを実行すると、仮想マシンが停止してリストから消えたことがわかる。

In [16]:
!ansible -b -a "virsh list" {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------



## dnsmasqの後始末

dnsmasqのリース情報の後始末。VM用IPアドレスが潤沢にある場合は不要。

In [17]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
1470675774 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.73 testvm004 *
1470674171 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.71 testvm002 *
1470665474 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.72 testvm003 *
1470673270 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.70 testvm001 *
1470675422 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.114 testvm007 *
1470665491 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.113 testvm006 *
1470673204 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.112 testvm005 *



In [18]:
for vmname, vmaddr in target_vmnames:
    !ansible -b -m lineinfile -a "dest=/var/lib/dnsmasq/dnsmasq.leases regexp='^.*\s+{ vmaddr }\s+.*' state=absent" {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.106 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}


In [19]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>




In [20]:
!ansible -b -m service -a "name=dnsmasq state=restarted" {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "changed": true, 
    "name": "dnsmasq", 
    "state": "started"
}


# Inventoryの更新

Inventoryから、破棄したマシンのIPアドレスを削除する。変更する前に、現在の内容をコピーしておく。

In [21]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmphESHPn'

In [22]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [23]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmphESHPn/inventory-old	2016-08-08 15:09:30.544364669 +0900
+++ inventory	2016-08-08 15:09:40.496448531 +0900
@@ -1,15 +1,6 @@
 [test-hadoop-hypervisor]
 XXX.XXX.XXX.106
 
-[test-hadoop-vms]
-XXX.XXX.XXX.70
-XXX.XXX.XXX.71
-XXX.XXX.XXX.72
-XXX.XXX.XXX.73
-XXX.XXX.XXX.112
-XXX.XXX.XXX.113
-XXX.XXX.XXX.114
-
 [docker_host]
 XXX.XXX.XXX.195
 XXX.XXX.XXX.196


削除したVMに**pingが通じない**かどうかを確認する。

In [24]:
!ansible -m ping {target_vm}

 [WARNING]: provided hosts list is empty, only localhost is available



# 後始末

一時ディレクトリを削除する。

In [25]:
!rm -fr {work_dir}